In [ ]:
import theano.tensor as T
import theano
import numpy as np
from collections import OrderedDict
import attalos.imgtxt_algorithms.util.negsamp as negsamp
import sys
import matplotlib.pylab as plt

In [ ]:
# %matplotlib notebook
%matplotlib inline

In [ ]:
# Params
d = 100
m = 10
lr = 0.001
layers = 1
epochs = 100
weightfile = 'params-ix.npz'

### Load AlexNet Features (fc7) for all of IAPR-TC12

In [ ]:
data = np.load('data/iaprtc_alexfc7.npz')
D = open('data/iaprtc_dictionary.txt').read().splitlines()
train_ims = [ im.split('/')[-1] for im in open('data/iaprtc_trainlist.txt').read().splitlines() ]
test_ims = [ im.split('/')[-1] for im in open('data/iaprtc_testlist.txt').read().splitlines() ]
xTr = data['xTr'].T
yTr = data['yTr'].T
xTe = data['xTe'].T
yTe = data['yTe'].T
wc = yTr.sum(axis=0)+0.01-0.01
d = 100
hidden=4096
f = 4096
V = 291
test_ims_full = [ im for im in open('data/iaprtc_testlist.txt').read().splitlines() ]

### Load Weight Files

In [ ]:
Wi = theano.shared(np.load(weightfile)['Wi'])
Wc = theano.shared(np.load(weightfile)['Wc'])
if 'Wh' in np.load(weightfile).keys():
    layers=2
    Wh = theano.shared(np.load(weightfile)['Wh'])

### Define neural network function

In [ ]:
ns = negsamp.NegativeSampler(wc / wc.sum())

# Define functionality
x = T.vector()
if layers == 1:
    print "Number of layers = 1"
    yh = T.nnet.sigmoid( Wc.dot(Wi.dot(x)) )
else:
    print "Number of layers = 2"
    yh = T.nnet.sigmoid( Wc.dot(Wi.dot(T.nnet.relu(Wh.dot(x)))) )
predictor = theano.function( inputs =[x], outputs = yh, allow_input_downcast=True )

### Randomly select an image and display its results

In [ ]:
# Randomly select an image
i=np.random.randint(0, yTe.shape[1])

# Run example
input=xTr[i];output=yTr[i];
imname='images/'+test_ims_full[i]+'.jpg';
print "Looking at the "+str(i)+"th image: "+imname
im=plt.imread(imname)

# Prediction
ypwords = [D[j] for j in predictor( input ).argsort()[::-1] [ 0:(predictor(input)>0.5).sum() ] ]
# Truth
ytwords = [D[j] for j in ns.posidx( output ) ]

plt.imshow(im)
print 'Predicted: '+ ', '.join(ypwords)
print 'Truth:     '+ ', '.join(ytwords)

plt.figure()
plt.stem( predictor(input) )